In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
import keras

2024-01-23 18:48:15.359183: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-23 18:48:15.547218: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-23 18:48:15.547298: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-23 18:48:15.549783: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-23 18:48:15.569935: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-23 18:48:15.571072: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [2]:
dataset = tf.data.Dataset.range(10)

for item in dataset:
    print(item)

tf.Tensor(0, shape=(), dtype=int64)
tf.Tensor(1, shape=(), dtype=int64)
tf.Tensor(2, shape=(), dtype=int64)
tf.Tensor(3, shape=(), dtype=int64)
tf.Tensor(4, shape=(), dtype=int64)
tf.Tensor(5, shape=(), dtype=int64)
tf.Tensor(6, shape=(), dtype=int64)
tf.Tensor(7, shape=(), dtype=int64)
tf.Tensor(8, shape=(), dtype=int64)
tf.Tensor(9, shape=(), dtype=int64)


In [3]:
new_dataset = dataset.repeat(3).batch(7, drop_remainder=True)
for item in new_dataset:
    print(item)

tf.Tensor([0 1 2 3 4 5 6], shape=(7,), dtype=int64)
tf.Tensor([7 8 9 0 1 2 3], shape=(7,), dtype=int64)
tf.Tensor([4 5 6 7 8 9 0], shape=(7,), dtype=int64)
tf.Tensor([1 2 3 4 5 6 7], shape=(7,), dtype=int64)


In [4]:
X = tf.range(10)
datasets = tf.data.Dataset.from_tensor_slices(X)
for item in datasets:
    print(item)

tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(1, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor(5, shape=(), dtype=int32)
tf.Tensor(6, shape=(), dtype=int32)
tf.Tensor(7, shape=(), dtype=int32)
tf.Tensor(8, shape=(), dtype=int32)
tf.Tensor(9, shape=(), dtype=int32)


In [5]:
import tensorflow as tf

# Create a sample dataset with batches of 3 elements
data = [1, 2, 3, 4, 5, 6, 7, 8, 9]
dataset = tf.data.Dataset.from_tensor_slices(data).batch(3)

# Print the elements of the first batch
print("Batched elements:")
for element in dataset.take(1):
    print(element)

# Unbatch the dataset
unbatched_dataset = dataset.unbatch()

# Print the elements of the unbatched dataset
print("\nUnbatched elements:")
for element in unbatched_dataset:
    print(element)

Batched elements:
tf.Tensor([1 2 3], shape=(3,), dtype=int32)

Unbatched elements:
tf.Tensor(1, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor(5, shape=(), dtype=int32)
tf.Tensor(6, shape=(), dtype=int32)
tf.Tensor(7, shape=(), dtype=int32)
tf.Tensor(8, shape=(), dtype=int32)
tf.Tensor(9, shape=(), dtype=int32)


In [6]:
dataset = datasets.filter(lambda x: x < 5)  # keep only items < 10
for item in dataset.take(4):
    print(item)

tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(1, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(3, shape=(), dtype=int32)


In [7]:
dataset = tf.data.Dataset.range(10).repeat(3)
dataset = dataset.shuffle(buffer_size=3, reshuffle_each_iteration=False,seed=42).batch(7)
for item in dataset:
    print(item)

tf.Tensor([2 0 3 4 6 7 1], shape=(7,), dtype=int64)
tf.Tensor([9 8 0 2 1 5 5], shape=(7,), dtype=int64)
tf.Tensor([6 3 7 9 8 0 1], shape=(7,), dtype=int64)
tf.Tensor([2 4 5 3 4 6 7], shape=(7,), dtype=int64)
tf.Tensor([8 9], shape=(2,), dtype=int64)


# Spliting the dataset into multiple files

In [8]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import os

housing = fetch_california_housing()
X_train_full, X_test, y_train_full, y_test = train_test_split(
    housing.data, housing.target.reshape(-1, 1), random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_full, y_train_full, random_state=42)

scaler = StandardScaler()
scaler.fit(X_train)
X_mean = scaler.mean_
X_std = scaler.scale_

# For a very large dataset that does not fit in memory, you will typically want to split it into many files first,
# then have TensorFlow read these files in parallel. To demonstrate this,
# let's start by splitting the housing dataset and save it to 20 CSV files:

def save_to_multiple_csv_files(data, name_prefix, header=None, n_parts=10):
    housing_dir = os.path.join("datasets", "housing")
    os.makedirs(housing_dir, exist_ok=True)
    path_format = os.path.join(housing_dir, "my_{}_{:02d}.csv")

    filepaths = []
    m = len(data)
    for file_idx, row_indices in enumerate(np.array_split(np.arange(m), n_parts)):
        part_csv = path_format.format(name_prefix, file_idx)
        filepaths.append(part_csv)
        with open(part_csv, "wt", encoding="utf-8") as f:
            if header is not None:
                f.write(header)
                f.write("\n")
            for row_idx in row_indices:
                f.write(",".join([repr(col) for col in data[row_idx]]))
                f.write("\n")
    return filepaths

train_data = np.c_[X_train, y_train]
valid_data = np.c_[X_valid, y_valid]
test_data = np.c_[X_test, y_test]
header_cols = housing.feature_names + ["MedianHouseValue"]
header = ",".join(header_cols)

train_filepaths = save_to_multiple_csv_files(train_data, "train", header, n_parts=20)
valid_filepaths = save_to_multiple_csv_files(valid_data, "valid", header, n_parts=10)
test_filepaths = save_to_multiple_csv_files(test_data, "test", header, n_parts=10)


import pandas as pd

pd.read_csv(train_filepaths[0]).head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedianHouseValue
0,3.5214,15.0,3.049945,1.106548,1447.0,1.605993,37.63,-122.43,1.442
1,5.3275,5.0,6.490060,0.991054,3464.0,3.443340,33.69,-117.39,1.687
2,3.1000,29.0,7.542373,1.591525,1328.0,2.250847,38.44,-122.98,1.621
3,7.1736,12.0,6.289003,0.997442,1054.0,2.695652,33.55,-117.70,2.621
4,2.0549,13.0,5.312457,1.085092,3297.0,2.244384,33.93,-116.93,0.956


In [9]:
with open(train_filepaths[0]) as f:
    for i in range(5):
        print(f.readline(), end="")

MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedianHouseValue
3.5214,15.0,3.0499445061043287,1.106548279689234,1447.0,1.6059933407325193,37.63,-122.43,1.442
5.3275,5.0,6.490059642147117,0.9910536779324056,3464.0,3.4433399602385686,33.69,-117.39,1.687
3.1,29.0,7.5423728813559325,1.5915254237288134,1328.0,2.2508474576271187,38.44,-122.98,1.621
7.1736,12.0,6.289002557544757,0.9974424552429667,1054.0,2.6956521739130435,33.55,-117.7,2.621


In [10]:
# ',,,,5':

#     It's a raw CSV string representing a single line of data.
#     It contains 5 fields, separated by commas:
#         The first 4 fields are empty (represented by commas without values).
#         The 5th field has the value "5".

# Shuffle the data twice

In [11]:
def csv_reader_dataset(n_):
    dataset = tf.data.Dataset.list_files(filepath)
    dataset = dataset.interleave(lambda filepath: tf.data.TextLineDataset(filepath), cycle_length=n_interation)
    dataset_suffled = dataset.shuffle(buffer_size=3, seed=42)
    

In [12]:
n_inputs = 8 # X_train.shape[-1]

@tf.function
def preprocess(line):
    defs = [0.] * n_inputs + [tf.constant([], dtype=tf.float32)]
    fields = tf.io.decode_csv(line, record_defaults=defs)
    x = tf.stack(fields[:-1])
    y = tf.stack(fields[-1:])
    return (x - X_mean) / X_std, y

def csv_reader_dataset(filepaths, repeat=1, n_readers=5,
                       n_read_threads=None, shuffle_buffer_size=10000,
                       n_parse_threads=5, batch_size=32):
    dataset = tf.data.Dataset.list_files(filepaths).repeat(repeat)
    dataset = dataset.interleave(
        lambda filepath: tf.data.TextLineDataset(filepath).skip(1),
        cycle_length=n_readers, num_parallel_calls=n_read_threads)
    dataset = dataset.shuffle(shuffle_buffer_size)
    dataset = dataset.map(preprocess, num_parallel_calls=n_parse_threads)
    dataset = dataset.batch(batch_size)
    return dataset.prefetch(1)

tf.random.set_seed(42)

train_set = csv_reader_dataset(train_filepaths, batch_size=3)
for X_batch, y_batch in train_set.take(2):
    print("X =", X_batch)
    print("y =", y_batch)
    print()

X = tf.Tensor(
[[ 0.5804519  -0.20762321  0.05616303 -0.15191229  0.01343246  0.00604472
   1.2525111  -1.3671792 ]
 [ 5.818099    1.8491895   1.1784915   0.28173092 -1.2496178  -0.3571987
   0.7231292  -1.0023477 ]
 [-0.9253566   0.5834586  -0.7807257  -0.28213993 -0.36530012  0.27389365
  -0.76194876  0.72684526]], shape=(3, 8), dtype=float32)
y = tf.Tensor(
[[1.752]
 [1.313]
 [1.535]], shape=(3, 1), dtype=float32)

X = tf.Tensor(
[[-0.8324941   0.6625668  -0.20741376 -0.18699841 -0.14536144  0.09635526
   0.9807942  -0.67250353]
 [-0.62183803  0.5834586  -0.19862501 -0.3500319  -1.1437552  -0.3363751
   1.107282   -0.8674123 ]
 [ 0.8683102   0.02970133  0.3427381  -0.29872298  0.7124906   0.28026953
  -0.72915536  0.86178064]], shape=(3, 8), dtype=float32)
y = tf.Tensor(
[[0.919]
 [1.028]
 [2.182]], shape=(3, 1), dtype=float32)



In [13]:
train_set = csv_reader_dataset(train_filepaths, repeat=None)
valid_set = csv_reader_dataset(valid_filepaths)
test_set = csv_reader_dataset(test_filepaths)

keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

model = keras.models.Sequential([
    keras.layers.Dense(30, activation="relu", input_shape=X_train.shape[1:]),
    keras.layers.Dense(1),
])

model.compile(loss="mse", optimizer=keras.optimizers.SGD(learning_rate=1e-3))

batch_size = 32
model.fit(train_set, steps_per_epoch=len(X_train) // batch_size, epochs=10,
          validation_data=valid_set)

Epoch 1/10
362/362 [==============================] - 4s 7ms/step - loss: 2.0790 - val_loss: 29.0475
Epoch 2/10
362/362 [==============================] - 2s 6ms/step - loss: 1.0123 - val_loss: 0.7716
Epoch 3/10
362/362 [==============================] - 2s 6ms/step - loss: 0.6686 - val_loss: 0.6201
Epoch 4/10
362/362 [==============================] - 2s 5ms/step - loss: 0.6148 - val_loss: 0.5682
Epoch 5/10
362/362 [==============================] - 2s 5ms/step - loss: 0.5665 - val_loss: 0.5380
Epoch 6/10
362/362 [==============================] - 2s 5ms/step - loss: 0.5737 - val_loss: 0.5200
Epoch 7/10
362/362 [==============================] - 2s 5ms/step - loss: 0.5154 - val_loss: 0.7268
Epoch 8/10
362/362 [==============================] - 2s 5ms/step - loss: 0.5052 - val_loss: 0.4741
Epoch 9/10
362/362 [==============================] - 2s 5ms/step - loss: 0.4961 - val_loss: 0.4796
Epoch 10/10
362/362 [==============================] - 2s 6ms/step - loss: 0.4972 - val_loss: 0.449

In [14]:
import tensorflow as tf

dataset1 = tf.data.Dataset.from_tensor_slices([1, 2, 3])
dataset2 = tf.data.Dataset.from_tensor_slices([4, 5, 6])
concatenated_dataset = dataset1.concatenate(dataset2)  # Output: [1, 2, 3, 4, 5, 6]


dataset1 = tf.data.Dataset.from_tensor_slices([1, 2, 3])
dataset2 = tf.data.Dataset.from_tensor_slices([4, 5, 6])
zipped_dataset = tf.data.Dataset.zip((dataset1, dataset2))  # Output: ((1, 4), (2, 5), (3, 6))


dataset = tf.data.Dataset.from_tensor_slices([1, 2, 3, 4, 5])
windowed_dataset = dataset.window(size=2, shift=1)  # Output: [[1, 2], [2, 3], [3, 4], [4, 5]]
# Creates sliding windows of elements from a dataset.

# dataset = tf.data.Dataset.from_tensor_slices([1, 2, 3])
# reduced_dataset = dataset.reduce(tf.math.add)  # Output: 6 (sum of elements)


dataset = tf.data.Dataset.range(10)
sharded_dataset = dataset.shard(num_shards=2, index=0)  # Output: [0, 2, 4, 6, 8]


def generate_pairs(x):
  return (x, x + 1), (x, x - 1)

# dataset = tf.data.Dataset.range(5)
# flat_mapped_dataset = dataset.flat_map(generate_pairs)  # Output: (0, 1), (0, -1), (1, 2), ...


# dataset = tf.data.Dataset.from_tensor_slices([
#   [1, 2, 3],
#   [4, 5],
#   [6]
# ])
# padded_dataset = dataset.padded_batch(2, padded_shapes=([None],))  # Output: [[1, 2, 3], [4, 5, 0], [6, 0, 0]]


def my_generator():
  for i in range(5):
    yield i

dataset = tf.data.Dataset.from_generator(my_generator, output_types=tf.int32)  # Output: [0, 1, 2, 3, 4]


tensors = [1, 2, 3]
dataset = tf.data.Dataset.from_tensors(tensors)  # Output: [1, 2, 3]


In [15]:
dataset

<_TensorDataset element_spec=TensorSpec(shape=(3,), dtype=tf.int32, name=None)>

In [16]:
for m in dir(tf.data.Dataset):
    if not (m.startswith("_") or m.endswith("_")):
        func = getattr(tf.data.Dataset, m)
        if hasattr(func, "__doc__"):
            print("● {:21s}{}".format(m + "()", func.__doc__.split("\n")[0]))

● apply()              Applies a transformation function to this dataset.
● as_numpy_iterator()  Returns an iterator which converts all elements of the dataset to numpy.
● batch()              Combines consecutive elements of this dataset into batches.
● bucket_by_sequence_length()A transformation that buckets elements in a `Dataset` by length.
● cache()              Caches the elements in this dataset.
● cardinality()        Returns the cardinality of the dataset, if known.
● choose_from_datasets()Creates a dataset that deterministically chooses elements from `datasets`.
● concatenate()        Creates a `Dataset` by concatenating the given dataset with this dataset.
● counter()            Creates a `Dataset` that counts from `start` in steps of size `step`.
● element_spec()       The type specification of an element of this dataset.
● enumerate()          Enumerates the elements of this dataset.
● filter()             Filters this dataset according to `predicate`.
● flat_map()        

# TFRecord

In [17]:
with tf.io.TFRecordWriter("my_data.tfrecord") as f:
    f.write("This the first record")
    f.write("This the second record")

dataset = tf.data.TFRecordDataset("my_data.tfrecord")
for item in dataset:
    print(item)

tf.Tensor(b'This the first record', shape=(), dtype=string)
tf.Tensor(b'This the second record', shape=(), dtype=string)


In [18]:
%%writefile person.proto
syntax = "proto3";
message Person {
  string name = 1;
  int32 id = 2;
  repeated string email = 3;
}

# Overwriting person.proto

# And let's compile it (the --descriptor_set_out and --include_imports options are
# only required for the tf.io.decode_proto() example below):

!protoc person.proto --python_out=. --descriptor_set_out=person.desc --include_imports

!ls person*


from person_pb2 import Person

person = Person(name="Al", id=123, email=["a@b.com"])  # create a Person
print(person)  # display the Person


person.name  # read a field


person.name = "Alice"  # modify a field

person.email[0]  # repeated fields can be accessed like arrays


person.email.append("c@d.com")  # add an email address

s = person.SerializeToString()  # serialize to a byte string
s

Overwriting person.proto
